In [ ]:
# from google.colab import drive
# drive.mount("/content/gdrive")
# BASE = "/content/gdrive/MyDrive/NLP_project/"
BASE = "../data"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
print(device)

cuda


In [ ]:
def load_data(path, include_neutral=True):
    data = pd.read_csv(path, delimiter="\t")
    data = data[data["gold_label"] != "-"]
    if not include_neutral:
        data = data[data["gold_label"] != "neutral"]
    premise = data["sentence1"].astype(str).tolist()
    hypothesis = data["sentence2"].astype(str).tolist()
    labels = data["gold_label"].astype(str).tolist()
    return premise, hypothesis, labels

In [ ]:
def labels_to_id(data):
    for i in range(len(data)):
        if data[i] == "entailment":
            data[i] = 0
        elif data[i] == "neutral":
            data[i] = 1
        else:
            data[i] = 2
    return data

In [ ]:
train_premise, train_hypothesis, train_labels = load_data(BASE + "snli_1.0_train.txt")
dev_premise, dev_hypothesis, dev_labels = load_data(BASE + "snli_1.0_dev.txt")
test_premise, test_hypothesis, test_labels = load_data(BASE + "snli_1.0_test.txt")

In [ ]:
train_labels = torch.tensor(labels_to_id(train_labels))
dev_labels = torch.tensor(labels_to_id(dev_labels))
test_labels = torch.tensor(labels_to_id(test_labels))

In [ ]:
model_name = "bert-base-cased"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def get_encodings(sentences):
    encodings = tokenizer(sentences, padding=True, truncation=True,
                          max_length=64, return_tensors="pt")
    return {
        "input_ids": encodings["input_ids"],
        "attention_mask": encodings["attention_mask"]
    }

In [ ]:
train_premise_encodings = get_encodings(train_premise)
train_hypothesis_encodings = get_encodings(train_hypothesis)
dev_premise_encodings = get_encodings(dev_premise)
dev_hypothesis_encodings = get_encodings(dev_hypothesis)
test_premise_encodings = get_encodings(test_premise)
test_hypothesis_encodings = get_encodings(test_hypothesis)

In [ ]:
class NLIDataset(Dataset):
    def __init__(self, premise, hypothesis, labels):
        self.premise = premise
        self.hypothesis = hypothesis
        self.labels = labels
    
    def __getitem__(self, idx):
        return (
            self.premise["input_ids"][idx], self.premise["attention_mask"][idx],
            self.hypothesis["input_ids"][idx], self.hypothesis["attention_mask"][idx],
            self.labels[idx]
            )
    
    def __len__(self):
        return len(self.labels)

In [ ]:
class NLIModel(nn.Module):
    def __init__(self, output_size):
        super(NLIModel, self).__init__()
        self.embedding = AutoModel.from_pretrained(model_name,
                                                   output_hidden_states=True).to(device)
        self.output_size = output_size
        self.feature_extractor = torch.hub.load("pytorch/vision:v0.10.0",
                                           "densenet121", pretrained=False).to(device)
        self.conv = nn.Conv2d(3072, 3, 1)
        self.output_layer = nn.Linear(1000, output_size)

    def get_embeddings(self, encoding):
        outputs = self.embedding(encoding["input_ids"],
                                 encoding["attention_mask"])
        hidden_states = outputs[2]
        embeddings = torch.cat(hidden_states[-1:-5:-1], dim=2)
        return embeddings
    
    def interaction(self, p, h):
        p = torch.unsqueeze(p, dim=2)
        h = torch.unsqueeze(h, dim=1)
        return p*h

    def forward(self, premise, hypothesis):
        p_embedding = self.get_embeddings(premise) # batch, p, d
        h_embedding = self.get_embeddings(hypothesis) # batch, h, d
        interaction_output = self.interaction(p_embedding, h_embedding)
        feature_extractor_input = self.conv(torch.permute(interaction_output, [0, 3, 1, 2]))
        features = self.feature_extractor(feature_extractor_input)
        return self.output_layer(features)

In [ ]:
def validate(model, dev_dataloader, loss_func):
    model.eval()
    with torch.no_grad():
        avg_loss = 0
        correct = 0
        total = 0
        for batch in dev_dataloader:
            p = {
                "input_ids": batch[0].to(device),
                "attention_mask": batch[1].to(device)
            }
            h = {
                "input_ids": batch[2].to(device),
                "attention_mask": batch[3].to(device)
            }
            y = batch[4].to(device)
            y_preds = model(p, h)
            preds = torch.argmax(y_preds, dim=1)
            loss = loss_func(y_preds, y)
            avg_loss += loss.item()
            correct += (preds == y).sum().item()
            total += y_preds.shape[0]
        avg_loss /= len(dev_dataloader)
        accuracy = correct / total
        return avg_loss, accuracy

In [ ]:
def train(model, train_dataloader, dev_dataloader, num_epochs, lr, save_path):
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    loss_func = nn.CrossEntropyLoss()
    num_batches = len(train_dataloader)
    one_fifth = num_batches // 30
    best_dev_loss, _ = validate(model, dev_dataloader, loss_func)
    for epoch in range(num_epochs):
        model.train()
        print(f"Epoch {epoch+1}/{num_epochs}:")
        avg_loss = 0
        correct = 0
        total = 0
        for i, batch in enumerate(train_dataloader):
            p = {
                "input_ids": batch[0].to(device),
                "attention_mask": batch[1].to(device)
            }
            h = {
                "input_ids": batch[2].to(device),
                "attention_mask": batch[3].to(device)
            }
            y = batch[4].to(device)
            y_preds = model(p, h)
            preds = torch.argmax(y_preds, dim=1)
            loss = loss_func(y_preds, y)
            avg_loss += loss.item()
            correct += (preds == y).sum().item()
            total += y_preds.shape[0]
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            if (i+1) % one_fifth == 0:
                print(f"Loss so far ({i+1}/{num_batches}):", avg_loss / (i+1))
                print("Accuracy:", correct / total)
        print("\nAverage Training Loss:", avg_loss / num_batches)
        print("Accuracy:", correct / total)
        dev_loss, dev_acc = validate(model, dev_dataloader, loss_func)
        if dev_loss < best_dev_loss:
            torch.save(model.state_dict(), save_path)
            best_dev_loss = dev_loss
        print("\nAverage Validation Loss:", dev_loss)
        print("Accuracy:", dev_acc)
        print()
    return model

In [ ]:
train_dataset = NLIDataset(train_premise_encodings, train_hypothesis_encodings,
                           train_labels)
dev_dataset = NLIDataset(dev_premise_encodings, dev_hypothesis_encodings,
                         dev_labels)
test_dataset = NLIDataset(test_premise_encodings, test_hypothesis_encodings,
                          test_labels)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=32)
test_dataloader = DataLoader(test_dataset, batch_size=32)

In [ ]:
model = NLIModel(3).to(device)
# model.load_state_dict(torch.load(BASE + "DIIN.pt", map_location=device))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
# test_loss, test_acc = validate(model, test_dataloader, nn.CrossEntropyLoss())

In [ ]:
# print(test_loss, test_acc)

0.5582170556344893 0.7894951140065146


In [ ]:
NUM_EPOCHS = 2
LR = 1e-5

In [ ]:
save_path = BASE + "DIIN.pt"
model = train(model, train_dataloader, dev_dataloader, NUM_EPOCHS, LR, save_path)

In [ ]:
loaded_model = NLIModel(3).to(device)
loaded_model.load_state_dict(torch.load(BASE + "DIIN.pt", map_location=device))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


<All keys matched successfully>

In [ ]:
test_labels = []
pred_labels = []
correct = 0
total = 0
loaded_model.eval()
label_types = [0, 1, 2]
confusion_matrix = {label_type: {label: 0 for label in label_types} for
                    label_type in label_types}
with torch.no_grad():
    for batch in test_dataloader:
        p = {
                "input_ids": batch[0].to(device),
                "attention_mask": batch[1].to(device)
        }
        h = {
                "input_ids": batch[2].to(device),
                "attention_mask": batch[3].to(device)
        }
        y = batch[4].to(device)
        y_preds = loaded_model(p, h)
        y_preds = torch.argmax(y_preds, dim=1)
        correct += (y_preds == y).sum().item()
        total += y_preds.shape[0]
        test_labels.extend(y.cpu().tolist())
        pred_labels.extend(y_preds.cpu().tolist())
        
accuracy = correct / total
with open("accuracy_cased.txt", "w") as file:
    file.write(f"{accuracy}")

In [ ]:
label_types = [0, 1, 2]
labels = ["Entailment", "Neutral", "Contradiction"]
confusion_matrix = {label_type: {label: 0 for label in label_types} for
                    label_type in label_types}
for i in range(len(test_labels)):
        confusion_matrix[pred_labels[i]][test_labels[i]] += 1
metrics = ("precision", "Recall", "F1-Score", "Support")
results = {label_type: {metric: 0 for metric in metrics} for label_type in label_types}
for Type in label_types:
        TP = confusion_matrix[Type][Type]
        FP = 0
        FN = 0
        for other in label_types:
            if Type != other:
                FP += confusion_matrix[Type][other]
                FN += confusion_matrix[other][Type]
        results[Type][metrics[0]] = TP / (TP + FP)
        results[Type][metrics[1]] = TP / (TP + FN)
        results[Type][metrics[2]] = (2 * results[Type][metrics[0]] * results[Type][metrics[1]]) /\
                                    (results[Type][metrics[0]] + results[Type][metrics[1]])
        results[Type][metrics[3]] = 0
        for other in label_types:
            results[Type][metrics[3]] += confusion_matrix[other][Type]
results_df = {}
for metric in metrics:
    results_df[metric] = []
    for label_type in label_types:
        results_df[metric].append(results[label_type][metric])
results_df = pd.DataFrame(results_df, index=labels)
results_df.to_csv("results_cased.csv")

In [ ]:
results_df

,precision,Recall,F1-Score,Support
Entailment,0.785714,0.916667,0.846154,12
Neutral,0.875000,0.777778,0.823529,9
Contradiction,0.800000,0.727273,0.761905,11


0.8125